# Dataset Overview for Bert tokenizer

In [ ]:
#Install essential packages
%%capture
!pip install datasets
!pip install transformers

!apt install git-lfs

In [ ]:
# Make sure your version of Transformers is at 4.11.0
# to run the following code correctly:
import datasets
import transformers

In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

##XSum

In [ ]:
# import dataset and metrics with huggingface
train_xsum = datasets.load_dataset("xsum", split="train")

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


In [ ]:
train_xsum

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 204045
})

In [ ]:
# Visualize the Data

import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    del df["id"]
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(train_xsum)

,document,summary
0,"The Londoners had four great chances to take the lead inside the first quarter of an hour, but had to wait until the second half to break the deadlock.\nMark Beevers opened the scoring as he headed in a corner from eight yards.\nVale's Enoch Andoh had chances to score either side of Beevers' goal before Lee Gregory hit a stoppage time penalty after he was fouled by Jak Alnwick.",Millwall moved up to 12th in League One after an impressive third away win of the season at Port Vale.
1,"Afridi and seven team-mates were punished after returning to their hotel in Sydney on Saturday 45 minutes late.\n""There are restaurants that have clubs inside them - nothing happened, we went for food then came back,"" all-rounder Afridi told BBC Urdu.\nThe group, including Ahmed Shehzad, were reportedly fined £150.\n""When you have no news to show, they spread these kind of rumours to get attention from people,"" added Afridi.\n""Nothing happened that night. We went for dinner - we are in Australia, not in [Pakistani cities] Thatta or Larkana,"" the all-rounder continued.\nPakistan take on neighbours and arch rivals India in Adelaide on Sunday.\nMedia playback is not supported on this device\nThey are looking to break a run that has seen them fail to win a World Cup match against India in five attempts, since their first game at the 1992 tournament, also held in Australia and New Zealand.\nPakistan won their final warm-up game against England by four wickets and Afridi - who will retire from ODIs after this year's World Cup - says his team are still looking to improve.\n""When you are playing in such a big event, where you are up against the best teams of the world, you always try to find the weaker points in yourself and you try to improve on them.\n""I wouldn't say that we are 100% prepared, but we are as ready as we will ever be.\n""Having said that, in such a big event you should be 100% ready, because cricket has become much more competitive than before. The other teams are all out to win.\n""Whatever your weak points are, whether it's batting, fielding or bowling, you work on these as soon as possible.""",Shahid Afridi has played down being handed a fine for breaching a Pakistan team curfew ahead of Sunday's World Cup match against India in Adelaide.
2,"UKIP wants full ""maritime sovereignty"" and Parliamentary supremacy over laws, no ""divorce bill"" nor payments to the EU budget after the UK's withdrawal.\nParty leader Paul Nuttall said UKIP would act as the ""guard dog"" of Brexit.\nThe prime minister will invoke Article 50 on Wednesday.\nThis will trigger a two-year process in which the UK and the remaining 27 EU nations will seek to agree the terms of the UK's exit as well as the outline of the UK's future relationship with the union.\nIn a speech on Monday, Mr Nuttall said that the UK's exit - set in motion by last year's Leave referendum vote - must be ""done and dusted"" by the end of 2019.\nAs it stands, the UK is on course to leave the EU on 29 March 2019, unless both sides agree to extend talks. But Brexit supporters have also spoken for the need for a completely ""clean break"", arguing there can be no transitional arrangements nor payments to maintain access to the single market afterwards.\nMr Nuttall set out six key objectives that he says any acceptable final agreement must be judged against:\n""While we will be willing Mrs May to succeed in agreeing to a full Brexit on our terms, we want her to keep faith with the will of the British people as expressed in the referendum result,"" he said.\n""We will also be letting her know that the political price she will pay for backsliding or watering down the ambitions will be very high indeed. I have described this role as akin to being the guard dog of Brexit.""\nUKIP has suggested fishing will be one of the early tests of how hard a bargain Mrs May is prepared to drive.\nIt has warned the government against adopting any aspects of Common Fisheries Policy into U

In [ ]:
# map article and summary len to dict as well as if sample is longer than 512 tokens
def map_to_length(x):
  x["article_len"] = len(tokenizer(x["document"]).input_ids)
  x["article_longer_512"] = int(x["article_len"] > 512)
  x["summary_len"] = len(tokenizer(x["summary"]).input_ids)
  x["summary_longer_64"] = int(x["summary_len"] > 64)
  x["summary_longer_128"] = int(x["summary_len"] > 128)
  return x

In [ ]:
# random choose 10% of the dataset
sample_size = 20000
data_stats = train_xsum.select(range(sample_size)).map(map_to_length, num_proc=4)

Token indices sequence length is longer than the specified maximum sequence length for this model (996 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (703 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1104 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (585 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
def compute_and_print_stats(x):
  if len(x["article_len"]) == sample_size:
    print(
        "Article Mean: {}, %-Articles > 512:{}, Summary Mean:{}, %-Summary > 64:{}, %-Summary > 128:{}".format(
            sum(x["article_len"]) / sample_size,
            sum(x["article_longer_512"]) / sample_size, 
            sum(x["summary_len"]) / sample_size,
            sum(x["summary_longer_64"]) / sample_size,
            sum(x["summary_longer_128"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats, 
  batched=True,
  batch_size=-1,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Article Mean: 478.99315, %-Articles > 512:0.34195, Summary Mean:28.0488, %-Summary > 64:0.00185, %-Summary > 128:0.0


From the Result, we can see that for random 20000 sample (10% of training dataset), our articles has average token length of 479, and 34% is longer than 512. And the mean summary's token length is 28, almost all are no longer than 64. 

`bert-base-cased` is limited to 512 tokens, which means we would have to cut possibly important information from the article. It should be reasonable to use **512** for tokenized article length and **64** for tokenized summary length on Bert-tokenizer for Xsum dataset.


##CNN/Daily

In [ ]:
train_cnn = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")

Downloading:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/159M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376M [00:00<?, ?B/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


In [ ]:
train_cnn

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [ ]:
show_random_elements(train_cnn)

In [ ]:
# map article and summary len to dict as well as if sample is longer than 512 tokens
def map_to_length(x):
  x["article_len"] = len(tokenizer(x["article"]).input_ids)
  x["article_longer_512"] = int(x["article_len"] > 512)
  x["summary_len"] = len(tokenizer(x["highlights"]).input_ids)
  x["summary_longer_64"] = int(x["summary_len"] > 64)
  x["summary_longer_128"] = int(x["summary_len"] > 128)
  return x

In [ ]:
sample_size = 25000
data_stats = train_cnn.select(range(sample_size)).map(map_to_length, num_proc=4)

Token indices sequence length is longer than the specified maximum sequence length for this model (1959 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1364 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (898 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1477 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
def compute_and_print_stats(x):
  if len(x["article_len"]) == sample_size:
    print(
        "Article Mean: {}, %-Articles > 512:{}, Summary Mean:{}, %-Summary > 64:{}, %-Summary > 128:{}".format(
            sum(x["article_len"]) / sample_size,
            sum(x["article_longer_512"]) / sample_size, 
            sum(x["summary_len"]) / sample_size,
            sum(x["summary_longer_64"]) / sample_size,
            sum(x["summary_longer_128"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats, 
  batched=True,
  batch_size=-1,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Article Mean: 848.47, %-Articles > 512:0.7356, Summary Mean:57.77368, %-Summary > 64:0.31812, %-Summary > 128:0.0


We can see that on average an article contains 848 tokens with 3/4 of the articles being longer than the model's `max_length` 512. The summary is on average 57 tokens long. Over 30% of our 25000-sample summaries are longer than 64 tokens, but none are longer than 128 tokens.

`bert-base-cased` is limited to 512 tokens. Because most of the important information is often found at the beginning of articles and because we want to be computationally efficient, we decide to stick to `bert-base-cased` with a `max_length` of 512 and 128 summary length for this dataset. This choice is not optimal but has shown to yield [good results](https://arxiv.org/abs/1907.12461) on CNN/Dailymail.

##Wikihow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/dataset

wikihowAll.csv


In [ ]:
# import dataset and metrics with huggingface
dataset_wiki = datasets.load_dataset("wikihow", "all", "/content/drive/MyDrive/dataset")

Using custom data configuration all-6f5101161f12f62f
Reusing dataset wikihow (/root/.cache/huggingface/datasets/wikihow/all-6f5101161f12f62f/1.2.0/5343fc81d685acaa086c9cc19eb8706206cd1f8b315792b04c1d7b92091c305e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train_wiki=dataset_wiki['train']
train_wiki

Dataset({
    features: ['text', 'headline', 'title'],
    num_rows: 157252
})

In [ ]:
# Visualize the Data

import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(train_wiki)

In [ ]:
# map article and summary len to dict as well as if sample is longer than 512 tokens
def map_to_length(x):
  x["article_len"] = len(tokenizer(x["text"]).input_ids)
  x["article_longer_512"] = int(x["article_len"] > 512)
  x["summary_len"] = len(tokenizer(x["headline"]).input_ids)
  x["summary_longer_64"] = int(x["summary_len"] > 64)
  x["summary_longer_128"] = int(x["summary_len"] > 128)
  return x

In [ ]:
sample_size = 20000
data_stats = train_wiki.select(range(sample_size)).map(map_to_length, num_proc=4)

Token indices sequence length is longer than the specified maximum sequence length for this model (752 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (715 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (557 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (786 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
def compute_and_print_stats(x):
  if len(x["article_len"]) == sample_size:
    print(
        "Article Mean: {}, %-Articles > 512:{}, Summary Mean:{}, %-Summary > 64:{}, %-Summary > 128:{}".format(
            sum(x["article_len"]) / sample_size,
            sum(x["article_longer_512"]) / sample_size, 
            sum(x["summary_len"]) / sample_size,
            sum(x["summary_longer_64"]) / sample_size,
            sum(x["summary_longer_128"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats, 
  batched=True,
  batch_size=-1,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Article Mean: 742.6638, %-Articles > 512:0.51815, Summary Mean:67.92995, %-Summary > 64:0.38625, %-Summary > 128:0.1109


We can see that on average an article contains 742 tokens with half of the articles being longer than the model's `max_length` 512. The summary is on average 68 tokens long. Over 30% of our 20000-sample summaries are longer than 64 tokens, but only some are longer than 128 tokens.

The dataset seems better than than cnn/daily when we use 512 for article length and 128 for summary length if these params can get good result on CNN/Daily. 